In [1]:
import os

os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/m.gnanachaithanya12/datascienceproject.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="m.gnanachaithanya12"
os.environ["MLFLOW_TRACKING_PASSWORD"]="36a1e6fcbaaa1278c825bd14fd08cbf88c1c5ec1"

In [2]:
%pwd

'd:\\Mlproject\\datascienceproject\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Mlproject\\datascienceproject'

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_path: Path
    all_params: dict
    target_column:str
    mlflow_uri:str

In [7]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml,create_directories,save_json

In [14]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path=CONFIG_PATH,
                 schema_file_path=SCHEMA_PATH,
                 params_file_path=PARAMS_PATH ):
        self.config = read_yaml(config_file_path)
        self.schema = read_yaml(schema_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_evauation_config(self)->ModelEvaluationConfig:
        config = self.config.model_evaluation
        schema = self.schema.TARGET_COLUMN
        params = self.params.ElasticNet

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            metric_file_path=config.metric_file_path,
            target_column=schema.name,
            all_params=params,
            mlflow_uri="https://dagshub.com/m.gnanachaithanya12/datascienceproject.mlflow"
        )
        return model_evaluation_config

In [15]:
import pandas as pd
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib


In [19]:
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):

        self.config = config
    
    def eval_metrics(self,actual,pred):
        rmse = np.sqrt(mean_squared_error(actual,pred))
        mae = mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)

        return rmse,mae,r2
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_path), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")
    

In [20]:
try:
    config = ConfigurationManager()
    modelevaluation_config = config.get_model_evauation_config()
    model_evaluation = ModelEvaluation(modelevaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-12-23 14:07:15,049: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-23 14:07:15,056: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-23 14:07:15,060: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-23 14:07:15,063: INFO: common: created directory at: artifacts]
[2024-12-23 14:07:15,066: INFO: common: created directory at: artifacts/model_evaluation]
[2024-12-23 14:07:15,587: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2024/12/23 14:07:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticnetModel'.
2024/12/23 14:07:40 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 1
Created version '1' of model 'ElasticnetModel'.


🏃 View run nosy-shoat-155 at: https://dagshub.com/m.gnanachaithanya12/datascienceproject.mlflow/#/experiments/0/runs/36d70f4f87124e3193861bcf4b17bc65
🧪 View experiment at: https://dagshub.com/m.gnanachaithanya12/datascienceproject.mlflow/#/experiments/0
